# Code

In [1]:
import pandas as pd
from pathlib import Path
import re
import glob
from datetime import datetime

In [ ]:
folder_path = Path(r'./data/')
file = list(folder_path.rglob('*Fleetpin*.csv'))
df = pd.read_csv(file[0], usecols=('Name', 'Odo / Hours / Hubo', 'Next Service Due', 'RUC Due', 'WoF Due', 'Rego Due'))


In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
# convert 'WoF Due' and 'Rego Due' data from string to datetime
df['WoF Due'] = pd.to_datetime(df['WoF Due'], dayfirst=True)
df['Rego Due'] = pd.to_datetime(df['Rego Due'], dayfirst=True)

# get today's date for comparison
today = datetime.now().date()
# subtract 'WoF Due' date from today's date to get days remaining for WOF Renewal
df['WOF Renew in days'] = (df['WoF Due'] - pd.Timestamp(today)).dt.days
df['Rego Renew in days'] = (df['Rego Due'] - pd.Timestamp(today)).dt.days


# cleaning data by removing characters from numbers using regex
df['Odo / Hours / Hubo'] = pd.Series(df['Odo / Hours / Hubo']).str.findall(r'\d+')
# get rid of list-like structure to strings where the elements are joined by comma
df['Odo / Hours / Hubo'] = [','.join(map(str, l)) for l in df['Odo / Hours / Hubo']]

df['ODO'] = df['Odo / Hours / Hubo'].apply(lambda x: x.split(',')[0])
df['Hubometer'] = df['Odo / Hours / Hubo'].apply(lambda x: x.split(',')[-1])

df['Next Service Due'] = pd.Series(df['Next Service Due']).str.strip('km')

df['RUC Due'] = pd.Series(df['RUC Due']).str.findall(r'\b\d+\b')
df['RUC Due']  = [','.join(map(str, l)) for l in df['RUC Due'] ]

df['Next Service Due'] = df['Next Service Due'].apply(lambda x: int(x))
df['RUC Due'] = df['RUC Due'].apply(lambda x: int(x))
df['ODO'] = df['ODO'].apply(lambda x: int(x))
df['Hubometer'] = df['Hubometer'].apply(lambda x: int(x))


df['RUC Due In Kms'] = df['RUC Due'] - df['Hubometer']
df['Service Due In Kms'] = df['Next Service Due'] - df['ODO']

df.head()

In [ ]:
df.dtypes

In [ ]:
# column re-ordering
df = df[['Name','ODO','Hubometer','Next Service Due','RUC Due','WoF Due','Rego Due','Service Due In Kms','WOF Renew in days','Rego Renew in days','RUC Due In Kms']]
# df = df.set_index('Name', drop=True)




# Report

In [ ]:
# columns_to_highlight = ['WOF Renew in days']


df.style\
    .highlight_between(subset='WOF Renew in days',left=0,right=30, color='lime')\
    .highlight_between(subset='WOF Renew in days',left=-1000,right=0, color='tomato')\
    .highlight_between(subset='RUC Due In Kms',left=0,right=5000, color='lime')\
    .highlight_between(subset='Service Due In Kms',left=0,right=2000, color='lime')\
    .highlight_between(subset='Service Due In Kms',left=-25000,right=0, color='tomato')\
    .highlight_between(subset='Rego Renew in days',left=0,right=30, color='lime')\
    .format({'RUC Due In Kms': '{0:,.0f} km', 'Service Due In Kms': '{0:,.0f} km'})
   
# def styling(x):
#     if x>0:
#         return "background-colr:lime;"

# df.style.applymap(styling, subset=["WOF Renew in days"])




In [ ]:
folder_path_report = Path(r'./report/')
file_path = folder_path_report / 'Report.csv'
df.to_csv(file_path)